# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [160]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [156]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# Columns Sex and fare Group by Encoding
sexfare_mean = df.groupby('Sex')['Fare'].mean().reset_index()
sexfare_max = df.groupby('Sex')['Fare'].max().reset_index()
sexfare_min = df.groupby('Sex')['Fare'].min().reset_index()
sexfare_median = df.groupby('Sex')['Fare'].median().reset_index()

sexfare_mode = df.groupby('Sex')['Fare'].apply(lambda x: x.mode()[0]).reset_index()


print(sexfare_mean)
print(sexfare_max)
print(sexfare_min)
print(sexfare_median)


      Sex       Fare
0  female  44.479818
1    male  25.523893
      Sex      Fare
0  female  512.3292
1    male  512.3292
      Sex  Fare
0  female  6.75
1    male  0.00
      Sex  Fare
0  female  23.0
1    male  10.5
Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


In [161]:
df_temp = pd.DataFrame()
print(df.columns)

df_temp = pd.merge(sexfare_mean, sexfare_max, how='left', on=['Sex'])
df_temp = pd.merge(df_temp, sexfare_min, how='left', on=['Sex'])
df_temp = pd.merge(df_temp, sexfare_median, how='left', on=['Sex'])
df_temp.columns = ['Sex','SexFare_Mean','SexFare_Max','SexFare_Min','SexFare_Median']
'''
sex_farecombine = [sexfare_mean] 
df_temp = pd.DataFrame(columns=sexfare_mean.columns)

#print(sex_farecombine)
print(df_temp)
for i  in sex_farecombine :
    print(f'i\n')
    print(f'{i}\n')
    df_temp = pd.merge(df_temp,i, on='Sex', how='right')
    print(f'df_temp\n')
    print(f'{df_temp}\n')
'''

#print(df_temp.columns)
df = df.merge(df_temp, on='Sex', how='right')
df.head()



Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexFare_Mean,SexFare_Max,SexFare_Min,SexFare_Median
0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,44.479818,512.3292,6.75,23.0
1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,512.3292,6.75,23.0
2,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44.479818,512.3292,6.75,23.0
3,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,44.479818,512.3292,6.75,23.0
4,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,44.479818,512.3292,6.75,23.0


In [162]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'SexFare_Mean', 'SexFare_Max', 'SexFare_Min', 'SexFare_Median']



,Pclass,Age,SibSp,Parch,Fare,SexFare_Mean,SexFare_Max,SexFare_Min,SexFare_Median
0,1,38.0,1,0,71.2833,44.479818,512.3292,6.75,23.0
1,3,26.0,0,0,7.9250,44.479818,512.3292,6.75,23.0
2,1,35.0,1,0,53.1000,44.479818,512.3292,6.75,23.0
3,3,27.0,0,2,11.1333,44.479818,512.3292,6.75,23.0
4,2,14.0,1,0,30.0708,44.479818,512.3292,6.75,23.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [169]:
# 原始特徵 + 邏輯斯迴歸
df_minus = df.drop(['SexFare_Mean','SexFare_Max','SexFare_Min','SexFare_Median'],axis=1)

train_X = MMEncoder.fit_transform(df_minus)

print(f'cross_vaildation{cross_val_score(LogisticRegression(),train_X,train_Y,cv=5).mean()}')


cross_vaildation0.6139162638880171


In [171]:
# 新特徵 + 邏輯斯迴歸


train_X = MMEncoder.fit_transform(df)


print(f'cross_vaildation{cross_val_score(LogisticRegression(),train_X,train_Y,cv=5).mean()}')

cross_vaildation0.615039859393635
